In [1]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["WANDB_DISABLED"] = "true"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

print("Installing packages...")
!pip install -q unsloth datasets transformers peft accelerate bitsandbytes
!pip install -q sentence-transformers sacrebleu
print("Packages installed!")

from unsloth import FastLanguageModel
import torch
from datasets import load_dataset, Dataset
from transformers import TrainingArguments, EarlyStoppingCallback
from unsloth import is_bfloat16_supported
from trl import SFTTrainer
from sentence_transformers import SentenceTransformer
import numpy as np
import re
from tqdm import tqdm
import random
from sacrebleu.metrics import BLEU

Installing packages...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.2/53.2 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.2/317.2 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.6/503.6 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.7/564.7 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.7/257.7 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.5/132.5 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.6/213.6 kB 18.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


🦥 Unsloth Zoo will now patch everything to make training faster!


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using

In [ ]:
class FastConfig:
    # Model
    model_name = "unsloth/Meta-Llama-3.1-8B-bnb-4bit"
    max_seq_length = 512
    target_samples = 2000        # Slightly more samples → better generalization
    train_split = 0.9

    # Training
    train_batch_size = 2         # Lower to fit GPU memory (T4 safe)
    gradient_accumulation_steps = 8  # Keeps effective batch size = 16
    learning_rate = 1.5e-4       # Slightly smaller → smoother updates
    max_steps = 600              # ~1 hour on Colab
    eval_steps = 100
    save_steps = 200
    logging_steps = 25
    warmup_steps = 100

    # LoRA
    lora_r = 64                  # Higher rank = better BLEU (but still fast)
    lora_alpha = 32
    lora_dropout = 0.1           # Keep low for stability
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj"
    ]

config = FastConfig()
print(f"Config loaded: {config.target_samples:,} samples, {config.max_steps} steps")


✅ Config loaded: 2,000 samples, 600 steps


In [3]:
def collect_dataset():
    all_data = []
    print("Loading opus-100 (Real parallel data)...")
    try:
        dataset = load_dataset("Helsinki-NLP/opus-100", "en-ta", split="train", streaming=True)
        for item in dataset:
            if len(all_data) >= config.target_samples // 2:
                break
            if 'translation' in item:
                ta = item['translation'].get('ta', '').strip()
                en = item['translation'].get('en', '').strip()
                if (5 < len(ta) < 300 and 5 < len(en) < 300 and
                    re.search(r'[\u0B80-\u0BFF]', ta)):
                    all_data.append({'tamil': ta, 'english': en})
        print(f"Collected {len(all_data):,} real translation pairs from opus-100")
    except Exception as e:
        print(f" Opus-100 failed: {e}")

    print("Generating synthetic data (Common phrases)...")
    templates = [
        ("வணக்கம்!", "Hello!"),
        ("நன்றி!", "Thank you!"),
        ("எப்படி இருக்கிறீர்கள்?", "How are you?"),
        ("நான் நன்றாக இருக்கிறேன்", "I am fine"),
        ("உங்கள் பெயர் என்ன?", "What is your name?"),
        ("எனக்கு உதவி வேண்டும்", "I need help"),
        ("இன்று வானிலை எப்படி இருக்கிறது?", "How is the weather today?"),
    ]

    while len(all_data) < config.target_samples:
        ta, en = templates[len(all_data) % len(templates)]
        all_data.append({'tamil': ta, 'english': en})

    print(f"Total: {len(all_data):,} samples (Real + Synthetic)")
    return all_data

raw_data = collect_dataset()
template = "Translate from Tamil to English.\n\nTamil: {tamil}\nEnglish: {english}"
formatted_texts = [template.format(**item) for item in raw_data]

dataset = Dataset.from_dict({"text": formatted_texts})
original_dataset = Dataset.from_dict({
    'tamil': [item['tamil'] for item in raw_data],
    'english': [item['english'] for item in raw_data]
})

print(f"Formatted {len(dataset):,} samples")

split = dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = split["train"]
eval_dataset = split["test"]

original_split = original_dataset.train_test_split(test_size=0.1, seed=42)
test_dataset_original = original_split["test"]

print(f"📊 Train: {len(train_dataset):,} | Test: {len(test_dataset_original):,}")

Loading opus-100 (Real parallel data)...


README.md: 0.00B [00:00, ?B/s]

Collected 1,000 real translation pairs from opus-100
Generating synthetic data (Common phrases)...
Total: 2,000 samples (Real + Synthetic)
Formatted 2,000 samples
📊 Train: 1,800 | Test: 200


In [4]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=config.model_name,
    max_seq_length=config.max_seq_length,
    dtype=None,
    load_in_4bit=True,
)

model = FastLanguageModel.get_peft_model(
    model,
    r=config.lora_r,
    target_modules=config.target_modules,
    lora_alpha=config.lora_alpha,
    lora_dropout=config.lora_dropout,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=42,
)

==((====))==  Unsloth 2025.10.1: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.1.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.10.1 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [5]:
training_args = TrainingArguments(
    output_dir="./tamil-translation-fast",
    per_device_train_batch_size=config.train_batch_size,
    per_device_eval_batch_size=config.train_batch_size,
    gradient_accumulation_steps=config.gradient_accumulation_steps,
    warmup_steps=config.warmup_steps,
    max_steps=config.max_steps,
    learning_rate=config.learning_rate,
    fp16=not is_bfloat16_supported(),
    bf16=is_bfloat16_supported(),
    logging_steps=25,
    optim="adamw_8bit",
    weight_decay=0.01,
    lr_scheduler_type="linear",
    seed=42,
    eval_strategy="steps",
    eval_steps=config.eval_steps,
    save_strategy="steps",
    save_steps=config.eval_steps,
    save_total_limit=2,
    load_best_model_at_end=True,
    report_to="none",
)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    dataset_text_field="text",
    max_seq_length=config.max_seq_length,
    args=training_args,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

trainer.train()

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/1800 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/200 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,800 | Num Epochs = 6 | Total steps = 600
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 8 x 1) = 16
 "-____-"     Trainable parameters = 167,772,160 of 8,198,033,408 (2.05% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,Validation Loss
100,0.646000,0.557120
200,0.559400,0.532458
300,0.413400,0.557381
400,0.269200,0.611226
500,0.172700,0.680965


Unsloth: Not an error, but LlamaForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for inst

TrainOutput(global_step=500, training_loss=0.4878980951309204, metrics={'train_runtime': 4787.6707, 'train_samples_per_second': 2.005, 'train_steps_per_second': 0.125, 'total_flos': 4.520670108288614e+16, 'train_loss': 0.4878980951309204, 'epoch': 4.426666666666667})

In [6]:
eval_results = trainer.evaluate()
print("\nFinal Training Metrics:")
print(f"  Eval Loss: {eval_results.get('eval_loss', 'N/A'):.4f}")

trainer.save_model("./tamil-translation-model")
tokenizer.save_pretrained("./tamil-translation-model")


Final Training Metrics:
  Eval Loss: 0.5325


('./tamil-translation-model/tokenizer_config.json',
 './tamil-translation-model/special_tokens_map.json',
 './tamil-translation-model/tokenizer.json')

In [7]:
sentence_model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")
bleu_metric = BLEU()
def translate(text):
    FastLanguageModel.for_inference(model)
    prompt = f"Translate from Tamil to English.\n\nTamil: {text}\nEnglish:"
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True).to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=128, temperature=0.1)
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return result.split("English:")[-1].strip().split("\n")[0] if "English:" in result else result
def compute_similarity(text1, text2):
    emb1 = sentence_model.encode(text1, convert_to_tensor=True).cpu().numpy()
    emb2 = sentence_model.encode(text2, convert_to_tensor=True).cpu().numpy()
    cos_sim = np.dot(emb1, emb2) / (np.linalg.norm(emb1) * np.linalg.norm(emb2) + 1e-8)
    return float(cos_sim)
print("\nQuick tests:")
test_cases = [
    "வணக்கம், நான் தமிழ் கற்கிறேன்",
    "உங்கள் பெயர் என்ன?",
    "நன்றி மிக்க நன்றி",
]

for i, tamil in enumerate(test_cases, 1):
    translation = translate(tamil)
    print(f"{i}. Tamil: {tamil}")
    print(f"   English: {translation}\n")
test_sample_size = getattr(config, "test_sample_size", 100)

print(f"\n🔍 Evaluating {test_sample_size} test samples...")
test_indices = random.sample(range(len(test_dataset_original)),
                            min(test_sample_size, len(test_dataset_original)))

similarities = []
high_quality = 0
semantic_matches = 0
exact_matches = 0

predictions = []
references = []

for idx in tqdm(test_indices, desc="Evaluating"):
    example = test_dataset_original[idx]
    tamil_text = example["tamil"]
    reference = example["english"]

    predicted = translate(tamil_text)

    sim = compute_similarity(predicted, reference)
    similarities.append(sim)

    if sim > 0.7:
        high_quality += 1
    if sim > 0.5:
        semantic_matches += 1
    if predicted.lower().strip() == reference.lower().strip():
        exact_matches += 1

    predictions.append(predicted)
    references.append([reference])

bleu_score = bleu_metric.corpus_score(predictions, references)

avg_sim = np.mean(similarities)
median_sim = np.median(similarities)
std_sim = np.std(similarities)
min_sim = np.min(similarities)
max_sim = np.max(similarities)

high_quality_pct = (high_quality / len(similarities)) * 100
semantic_pct = (semantic_matches / len(similarities)) * 100
exact_match_pct = (exact_matches / len(similarities)) * 100


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]


Quick tests:
1. Tamil: வணக்கம், நான் தமிழ் கற்கிறேன்
   English: Hello, I am learning Tamil.

2. Tamil: உங்கள் பெயர் என்ன?
   English: What is your name?

3. Tamil: நன்றி மிக்க நன்றி
   English: Thank you very much.


🔍 Evaluating 100 test samples...


Evaluating: 100%|██████████| 100/100 [02:06<00:00,  1.26s/it]


In [8]:
print(f"\n{'='*70}")
print("COMPREHENSIVE EVALUATION RESULTS")
print(f"{'='*70}")
print(f"\nBLEU Score:")
print(f"{'─'*70}")
print(f"  BLEU:           {bleu_score.score:.2f}")
print(f"  BLEU-1:         {bleu_score.precisions[0]:.2f}")
print(f"  BLEU-2:         {bleu_score.precisions[1]:.2f}")
print(f"  BLEU-3:         {bleu_score.precisions[2]:.2f}")
print(f"  BLEU-4:         {bleu_score.precisions[3]:.2f}")
print(f"  Brevity Penalty: {bleu_score.bp:.4f}")
print(f"\nBERT Semantic Similarity:")
print(f"{'─'*70}")
print(f"  Average:        {avg_sim:.4f}")
print(f"  Median:         {median_sim:.4f}")
print(f"  Std Dev:        {std_sim:.4f}")
print(f"  Min:            {min_sim:.4f}")
print(f"  Max:            {max_sim:.4f}")
print(f"\nTranslation Quality:")
print(f"{'─'*70}")
print(f"  Excellent (BERT>0.7):  {high_quality:3d}/{len(similarities)} ({high_quality_pct:5.1f}%)")
print(f"  Good (BERT>0.5):       {semantic_matches:3d}/{len(similarities)} ({semantic_pct:5.1f}%)")
print(f"  Exact Matches:         {exact_matches:3d}/{len(similarities)} ({exact_match_pct:5.1f}%)")
print(f"\nTraining Metrics:")
print(f"{'─'*70}")
print(f"  Final Eval Loss:       {eval_results.get('eval_loss', 'N/A'):.4f}")
print(f"  Training Samples:      {len(train_dataset):,}")
print(f"  Test Samples Evaluated: {len(similarities)}")
print(f"\nOverall Performance Score:")
print(f"{'─'*70}")
overall_score = (bleu_score.score / 100 * 0.4) + (avg_sim * 0.6)
print(f"  Combined Score:        {overall_score:.4f} (BLEU 40% + BERT 60%)")
quality_rating = " Excellent" if overall_score > 0.7 else " Good" if overall_score > 0.5 else "⚠ Fair" if overall_score > 0.3 else " Needs Improvement"
print(f"  Quality Rating:        {quality_rating}")
print(f"\n{'='*70}")
print("Model ready at: ./tamil-translation-model")
print("Training and evaluation complete!")
print(f"{'='*70}")


COMPREHENSIVE EVALUATION RESULTS

BLEU Score:
──────────────────────────────────────────────────────────────────────
  BLEU:           63.89
  BLEU-1:         100.00
  BLEU-2:         66.67
  BLEU-3:         50.00
  BLEU-4:         50.00
  Brevity Penalty: 1.0000

BERT Semantic Similarity:
──────────────────────────────────────────────────────────────────────
  Average:        0.8349
  Median:         0.9822
  Std Dev:        0.2377
  Min:            0.0434
  Max:            1.0000

Translation Quality:
──────────────────────────────────────────────────────────────────────
  Excellent (BERT>0.7):   75/100 ( 75.0%)
  Good (BERT>0.5):        87/100 ( 87.0%)
  Exact Matches:          45/100 ( 45.0%)

Training Metrics:
──────────────────────────────────────────────────────────────────────
  Final Eval Loss:       0.5325
  Training Samples:      1,800
  Test Samples Evaluated: 100

Overall Performance Score:
──────────────────────────────────────────────────────────────────────
  Combined 

In [9]:
results_summary = f"""
TAMIL-ENGLISH TRANSLATION MODEL EVALUATION REPORT
{'='*70}

BLEU METRICS:
  - BLEU Score: {bleu_score.score:.2f}
  - BLEU-1: {bleu_score.precisions[0]:.2f}
  - BLEU-2: {bleu_score.precisions[1]:.2f}
  - BLEU-3: {bleu_score.precisions[2]:.2f}
  - BLEU-4: {bleu_score.precisions[3]:.2f}
  - Brevity Penalty: {bleu_score.bp:.4f}

BERT SEMANTIC SIMILARITY:
  - Average: {avg_sim:.4f}
  - Median: {median_sim:.4f}
  - Std Dev: {std_sim:.4f}
  - Range: {min_sim:.4f} - {max_sim:.4f}

TRANSLATION QUALITY:
  - Excellent (>0.7): {high_quality_pct:.1f}%
  - Good (>0.5): {semantic_pct:.1f}%
  - Exact Matches: {exact_match_pct:.1f}%

TRAINING INFO:
  - Final Eval Loss: {eval_results.get('eval_loss', 'N/A'):.4f}
  - Training Samples: {len(train_dataset):,}
  - Test Samples: {len(similarities)}

OVERALL PERFORMANCE:
  - Combined Score: {overall_score:.4f}
  - Rating: {quality_rating}

Model saved at: ./tamil-translation-model
"""

with open("./evaluation_results.txt", "w", encoding="utf-8") as f:
    f.write(results_summary)

print("\n Results saved to: ./evaluation_results.txt")


 Results saved to: ./evaluation_results.txt
